In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing ##Parallelization library
from multiprocessing import Pool ## Parallelization function
import time
import igraph as ig

#### Expression data importation

In [ ]:
data=pd.read_csv('~/Projects/datos_camilo/Data/Expression/TPM.At.ATLAS.txt',sep='\t')
data=data.drop(['Strand','Length'],axis=1)

### First scaling data

Here the expression data is filtered by level expression sample if the count is bigger than 6

In [ ]:
dat=(data.to_numpy()[:,1:]>=2)*1
data['count']=dat.sum(axis=1)
data=data[data['count']>=6]
data=data.drop(['count'],axis=1)

The expression result data is scaled using log2

In [ ]:
data_scaled=data.drop(['GeneID'],axis=1)
data_scaled[data_scaled.columns]=np.log2(data_scaled.to_numpy()+1) ## Here es explicit the operation
data_scaled.insert(0,'GeneID',data['GeneID']) #Here it's inserted again the ID of genes
data_scaled=data_scaled.reset_index()

## Kmeans method

In [ ]:
## Kmeans library is importated 
from sklearn.cluster import KMeans

## Taking the transpose of the scaled data after to use log2
X=data_scaled.to_numpy()[:,2:].T

np.shape(X)


kmeans=KMeans(n_clusters=35).fit(X) ## Calculating the kmeans whith k=35
global centroides

## Centroides has all clusters from kmeans whith shape (# of genes,# of clusters)
centroides=kmeans.cluster_centers_.T 

##Each columns es the level expression of the gene regard the corresponding cluster and here each row is standardized
centroides=preprocessing.scale(centroides,axis=1)
Centroides=pd.DataFrame(centroides) ##Define a dataframe whith the new cluster data standardized
Centroides['GeneID']=data_scaled['GeneID']

print('Ya terminó de calcular los centroides')

## Computing the euclidian distance between the genes

Define a function will calculate the euclidian distance feature vector between two genes using the Centroides Dataframe

In [ ]:
from scipy.spatial import distance_matrix ##Calculate the matrix distance among a given points set.

def distance_genes(gene):
    gene1=Centroides[Centroides['GeneID']==g.vs['name'][gene[0]]].to_numpy()[0,:34]
    gene2=Centroides[Centroides['GeneID']==g.vs['name'][gene[1]]].to_numpy()[0,:34]
    nodes=np.array([gene1,gene2])
    nodes=nodes.reshape((-1,2))
    distance=np.triu(distance_matrix(nodes,nodes),1)
    distance=distance.reshape((1,-1))
    distance=distance[distance != 0]
    
    return distance

In [ ]:
## The factor transcription list is imported
TFs=pd.read_csv('/maindisk/fabio/Projects/datos_camilo/Data/At_TFs_iTAK.txt', sep='\t', header=None)

### Actions

In this section the euclidian distance feature vector for every TF from the list imported is computed. For that a For loop is used to pass throught all TF list, and another parallelizise For loop whit Pool function is used to compute the euclidian distance feature vector for the corresponfing TF.

In [1]:
size=len(TFs[0])
l=0
for i in TFs[0][0:]:
    
    place=Centroides['GeneID']==i
    bool=True in np.array(place)
    print(bool)
    if (bool==True):
        
        edges_prove=[]
        for j in range(len(data_scaled)):
            edges_prove.append([i, data_scaled['GeneID'][j]])
        edges_porve=np.array(edges_prove)
        
        g=ig.Graph()
        g.add_vertices(data_scaled['GeneID'])
        g.add_edges(edges_prove)
        edges=g.get_edgelist()
        
        pool=Pool()
        start_time=time.time()
        result=pool.map(distance_genes,edges)
        end_time=time.time()-start_time
        pool.close()
        pool.join()
        t=end_time
    
        np.savetxt('/maindisk/fabio/Projects/datos_camilo/TF_feature_distance/'+i+'.txt',result,fmt='%s')
    l+=1
    print('Van '+ str(l) +' de '+str(size)+' en ', t)
print('El proceso ha finalizado')

Ya terminó de calcular los centroides
True
Van 1 de 2207 en  9.298948764801025
True
Van 2 de 2207 en  9.546540021896362
True
Van 3 de 2207 en  9.70131516456604
True
Van 4 de 2207 en  9.856417655944824
True
Van 5 de 2207 en  8.122629880905151
True
Van 6 de 2207 en  6.352393388748169
True
Van 7 de 2207 en  10.133856058120728
True
Van 8 de 2207 en  19.95345902442932
True
Van 9 de 2207 en  8.17276644706726
True
Van 10 de 2207 en  9.80226445198059
True
Van 11 de 2207 en  9.151124715805054
True
Van 12 de 2207 en  9.657667636871338
True
Van 13 de 2207 en  8.021199226379395
True
Van 14 de 2207 en  6.853055953979492
True
Van 15 de 2207 en  7.007967233657837
True
Van 16 de 2207 en  7.070651054382324
False
Van 17 de 2207 en  7.070651054382324
True
Van 18 de 2207 en  7.3309032917022705
True
Van 19 de 2207 en  6.981106519699097
False
Van 20 de 2207 en  6.981106519699097
True
Van 21 de 2207 en  8.065845727920532
False
Van 22 de 2207 en  8.065845727920532
True
Van 23 de 2207 en  9.805740356445312
Tru

Van 193 de 2207 en  6.918578147888184
True
Van 194 de 2207 en  7.1228086948394775
True
Van 195 de 2207 en  7.34136700630188
True
Van 196 de 2207 en  7.157793760299683
True
Van 197 de 2207 en  7.652902364730835
True
Van 198 de 2207 en  7.62939715385437
True
Van 199 de 2207 en  8.394191980361938
True
Van 200 de 2207 en  11.331875801086426
True
Van 201 de 2207 en  7.447488784790039
True
Van 202 de 2207 en  7.543627977371216
True
Van 203 de 2207 en  9.186623096466064
True
Van 204 de 2207 en  7.36014199256897
True
Van 205 de 2207 en  9.485387802124023
True
Van 206 de 2207 en  9.124360084533691
True
Van 207 de 2207 en  9.489060163497925
True
Van 208 de 2207 en  9.34264874458313
True
Van 209 de 2207 en  7.910711050033569
True
Van 210 de 2207 en  9.429288625717163
False
Van 211 de 2207 en  9.429288625717163
True
Van 212 de 2207 en  7.208070278167725
True
Van 213 de 2207 en  7.3677308559417725
True
Van 214 de 2207 en  7.116243600845337
True
Van 215 de 2207 en  8.704620838165283
True
Van 216 de 

Van 383 de 2207 en  13.128024101257324
True
Van 384 de 2207 en  7.102351188659668
True
Van 385 de 2207 en  7.476372241973877
True
Van 386 de 2207 en  6.798213243484497
True
Van 387 de 2207 en  9.71639347076416
True
Van 388 de 2207 en  8.068558692932129
True
Van 389 de 2207 en  7.467427492141724
True
Van 390 de 2207 en  7.3356194496154785
True
Van 391 de 2207 en  7.397003650665283
True
Van 392 de 2207 en  7.2920496463775635
True
Van 393 de 2207 en  7.300832271575928
True
Van 394 de 2207 en  7.313608884811401
True
Van 395 de 2207 en  7.4639482498168945
True
Van 396 de 2207 en  13.931408405303955
False
Van 397 de 2207 en  13.931408405303955
True
Van 398 de 2207 en  10.2446870803833
True
Van 399 de 2207 en  7.76728081703186
True
Van 400 de 2207 en  7.268197536468506
False
Van 401 de 2207 en  7.268197536468506
True
Van 402 de 2207 en  7.145257234573364
False
Van 403 de 2207 en  7.145257234573364
True
Van 404 de 2207 en  7.759037733078003
True
Van 405 de 2207 en  7.4612932205200195
True
Van 

Van 574 de 2207 en  7.3067872524261475
True
Van 575 de 2207 en  6.807068347930908
True
Van 576 de 2207 en  7.531001091003418
True
Van 577 de 2207 en  6.936072587966919
True
Van 578 de 2207 en  7.474947214126587
True
Van 579 de 2207 en  12.472169160842896
True
Van 580 de 2207 en  8.756727457046509
True
Van 581 de 2207 en  9.629613876342773
True
Van 582 de 2207 en  7.742428779602051
True
Van 583 de 2207 en  9.585488319396973
True
Van 584 de 2207 en  8.848592281341553
True
Van 585 de 2207 en  7.093270540237427
True
Van 586 de 2207 en  7.512272834777832
False
Van 587 de 2207 en  7.512272834777832
True
Van 588 de 2207 en  8.617066383361816
False
Van 589 de 2207 en  8.617066383361816
True
Van 590 de 2207 en  9.510083675384521
False
Van 591 de 2207 en  9.510083675384521
True
Van 592 de 2207 en  7.30890154838562
True
Van 593 de 2207 en  7.78455114364624
True
Van 594 de 2207 en  7.493301630020142
True
Van 595 de 2207 en  7.350100755691528
True
Van 596 de 2207 en  8.405028104782104
True
Van 597 

Van 765 de 2207 en  6.933817386627197
True
Van 766 de 2207 en  7.98611044883728
True
Van 767 de 2207 en  8.035261154174805
True
Van 768 de 2207 en  7.324077367782593
True
Van 769 de 2207 en  9.744672298431396
True
Van 770 de 2207 en  6.910273551940918
True
Van 771 de 2207 en  8.908771753311157
True


KeyboardInterrupt: 